In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-p9gan9p5
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-p9gan9p5
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=9df011698d3c07172beea3fb552824ed712f43f23003f378cf1d3b0664cc7604
  Stored in directory: /tmp/pip-ephem-wheel-cache-5cednlcc/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include<stdio.h>
#include<cuda.h>
#include<stdlib.h>
#include<time.h>

#define N 500

__global__ void add(int *a, int *b, int *c){

int tid = threadIdx.x;
if (tid < N){
	c[tid]= a[tid] + b[tid];
	}

}

 int main (void){
        int arraySize = 6;
    int a[arraySize] = { 1, 2, 3, 4, 5, 0 };
    int b[arraySize] = { 10, 20, 30, 40, 50, 1 };
    int c[arraySize] = {0};
    printf("arr 1: ", a);
    printf("arr 2: ", b);
/*int a[N], b[N], c[N];*/
int *dev_a, *dev_b,*dev_c;
cudaError_t err=cudaSuccess;
err=cudaMalloc((void**)&dev_a,N * sizeof(int)); 
if (err !=cudaSuccess)
{ 	printf("failed to  allocate on device \n");
	printf("error is:\n",cudaGetErrorString(err));
exit(EXIT_FAILURE);
}
cudaMalloc((void**)&dev_b,N * sizeof(int)); 
cudaMalloc((void**)&dev_c,N * sizeof(int)); 

for(int i=0;i<6;i++){
a[i] =i;
b[i] = i*i;

c[i]=0;
}

/*for(int i=0;i<6;i++){
printf(" c conttents are =%d\n",  c[i]);
} */
cudaEvent_t start, end;
cudaEventCreate(&start);
cudaEventCreate(&end);
cudaEventRecord(start);

cudaMemcpy(dev_a,a,N*sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(dev_b,b,N*sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(dev_c,c,N*sizeof(int), cudaMemcpyHostToDevice);
add<<<1,N>>>(dev_a,dev_b,dev_c);

err=cudaMemcpy(c,dev_c,N*sizeof(int), cudaMemcpyDeviceToHost);
if (err !=cudaSuccess)
{ 	printf("failed to  copy from device \n");
exit(EXIT_FAILURE);
}

cudaEventRecord(end);
cudaEventSynchronize(end);
float time = 0;
cudaEventElapsedTime(&time, start, end);
printf("execution time=%f\n",time);

for(int i=0;i<N;i++){
//printf("%d +%d=%d\n", a[i], b[i], c[i]);
}

cudaFree(dev_a);
cudaFree(dev_b);
cudaFree(dev_c);

return 0;
}


UsageError: Cell magic `%%cu` not found.
